In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import datatable as dt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()
df_samples = df.sample(n=100000,replace=False)
df_samples = df_samples.astype({c: np.float32 for c in df_samples.select_dtypes(include='float64').columns})

In [ ]:
df_samples = df_samples.query('weight>0').reset_index(drop=True)
df_samples['action'] = (df_samples['resp']>0).astype(int)
df_samples['weight_t'] = abs(df_samples['resp'])*df_samples['weight'].transform('sqrt')

In [ ]:
mean = df_samples.mean()
df_samples = df_samples.fillna(mean)

In [ ]:
features = [c for c in df_samples.columns if 'feature' in c]
#X = df_samples.loc[:, df_samples.columns.str.contains('feature')]
#y = df_samples.loc[:, 'action']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


In [ ]:
xgb = xgboost.XGBClassifier(n_estimators=200,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=2020)
xgb_demo = xgb.fit(df_samples[features], df_samples['action'],sample_weight=df_samples['weight_t'])
#y_pred_boost = xgb_demo.predict(X_test)

In [ ]:
import janestreet
from tqdm.notebook import tqdm
janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test: 
    X_test = test_df.loc[:, features]
    mean = df_samples.mean()
    X_test = X_test.fillna(mean)
    sample_prediction_df.action = xgb_demo.predict(X_test)
    env.predict(sample_prediction_df)